In [1]:
from comet_ml import Experiment

#create an experiment with your api key
experiment = Experiment(api_key="KYvcpL63EabnBbmTC2PzoWWz6",
                        project_name='mnist',
                        workspace="ksaaskil",
                        auto_param_logging=False)



COMET WARNING: We detected that you are running inside a Ipython/Jupyter notebook environment but we cannot save your notebook source code. Please be sure to have installed comet_ml as a notebook server extension by running:
jupyter comet_ml enable 
For more details, please refer to: https://www.comet.ml/docs/python-sdk/warnings-errors
COMET WARNING: Comet.ml support for Ipython Notebook is limited at the moment, automatic monitoring and stdout capturing is deactivated 
For more details, please refer to: https://www.comet.ml/docs/python-sdk/warnings-errors
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/ksaaskil/mnist/46afd47c1bb9403f93a0b077ba66e225



In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

batch_size = 128
num_classes = 10
epochs = 20
num_nodes = 64
optimizer = 'adam'
activation = 'relu'

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#these will all get logged 
params={'batch_size':batch_size,
        'epochs':epochs,
        'layer1_type':'Dense',
        'layer1_num_nodes':num_nodes,
        'layer1_activation':activation,
        'optimizer':optimizer
}
model = Sequential()
model.add(Dense(num_nodes, activation='relu', input_shape=(784,)))
#model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

#print model.summary() to preserve automatically in `Output` tab
print(model.summary())

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

#will log metrics with the prefix 'train_'
with experiment.train():
    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test),
                        callbacks=[EarlyStopping(monitor='val_loss', min_delta=1e-4,patience=3, verbose=1, mode='auto')])

#will log metrics with the prefix 'test_'
with experiment.test():
    loss, accuracy = model.evaluate(x_test, y_test)
    metrics = {
        'loss':loss,
        'accuracy':accuracy
    }
    experiment.log_multiple_metrics(metrics)

experiment.log_multiple_params(params)
experiment.log_dataset_hash(x_train) #creates and logs a hash of your data

Using TensorFlow backend.


60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 1s 14us/step - loss: 0.4230 - acc: 0.8854 - val_loss: 0.2229 - val_acc: 0.9360
Epoch 2/20
60000/60000 [==============================] - 1s 11us/step - loss: 0.2000 - acc: 0.9441 - val_loss: 0.1768 - val_acc: 0.9479
Epoch 3/20
60000/60000 [==============================] - 1s 11us/step - loss: 0.1557 - acc: 0.9555 - val_loss: 0.1428 - val_acc: 0.9598
Epoch 4/20
60000/60000 [=====